In [141]:
# Зависимости
import requests
import pandas as pd

In [167]:
BASE_URL = "https://iss.moex.com/iss/"

In [168]:
def get_ofz_securities(i: int) -> tuple[pd.DataFrame, int]:
  """ get_securities
  Функция для получения списка ОФЗ фондового рынка облигаций.

  Args:
    i - индекс страницы.

  Returns:
    tuple(pd.DataFrame, int)
    Таблица облигаций и размер таблицы.
  """
  response_securities = requests.get(
      url=BASE_URL + "securities.json",
      params={
          # Фондовый рынок
          "engine": "stock",
          # Рынок облигаций
          "market": "bonds",
          # Облигации Федерального займа
          "q": "ОФЗ",
          # Индекс страницы
          "start": i * 100
      }
  )

  response_status_code = response_securities.status_code

  if response_status_code == 200:
    securities = response_securities.json()["securities"]
    securities_data = pd.DataFrame(data=securities["data"], columns=securities["columns"])
  else:
    securities_data = pd.DataFrame()

  return securities_data, securities_data.shape[0]

In [169]:
i = 0  # индекс стартовой страницы
securities = []
while True:
  # Собираем полный список ОФЗ, торгуемых на Московской бирже
  securities_sample, securities_size = get_ofz_securities(i)
  if securities_size == 0:
    break

  securities.append(securities_sample)
  i += 1

In [170]:
# Объединяем полученные данные
securities_data = pd.concat(securities).reset_index(drop=True)

In [171]:
display(securities_data.head())

,secid,shortname,regnumber,name,isin,is_traded,emitent_id,emitent_title,emitent_inn,emitent_okpo,type,group,primary_boardid,marketprice_boardid
0,SU26238RMFS4,ОФЗ 26238,26238RMFS,ОФЗ-ПД 26238 15/05/2041,RU000A1038V6,1,1228,Министерство финансов Российской Федерации,7710168360,None,ofz_bond,stock_bonds,TQOB,TQOB
1,SU26248RMFS3,ОФЗ 26248,26248RMFS,ОФЗ-ПД 26248 16/05/40,RU000A108EH4,1,1228,Министерство финансов Российской Федерации,7710168360,None,ofz_bond,stock_bonds,TQOB,TQOB
2,SU26247RMFS5,ОФЗ 26247,26247RMFS,ОФЗ-ПД 26247 11/05/39,RU000A108EF8,1,1228,Министерство финансов Российской Федерации,7710168360,None,ofz_bond,stock_bonds,TQOB,TQOB
3,SU26243RMFS4,ОФЗ 26243,26243RMFS,ОФЗ-ПД 26243 19/05/38,RU000A106E90,1,1228,Министерство финансов Российской Федерации,7710168360,None,ofz_bond,stock_bonds,TQOB,TQOB
4,SU26233RMFS5,ОФЗ 26233,26233RMFS,ОФЗ-ПД 26233 18/07/2035,RU000A101F94,1,1228,Министерство финансов Российской Федерации,7710168360,None,ofz_bond,stock_bonds,TQOB,TQOB


In [172]:
securities_data["emitent_title"].unique()

array(['Министерство финансов Российской Федерации',
       'Публичное акционерное общество "Сбербанк России"'], dtype=object)

In [173]:
# Исключаем всех эмитентов, кроме МинФин РФ
securities_data = securities_data.loc[securities_data["emitent_title"] != 'Публичное акционерное общество "Сбербанк России"'].reset_index(drop=True)

In [174]:
# Исключаем ненужные поля
use_cols_securities_data = ["secid", "shortname", "regnumber", "isin", "is_traded", "group", "marketprice_boardid"]
securities_data = securities_data[use_cols_securities_data]

In [189]:
def get_security_info(secid: str) -> pd.DataFrame:
  """ get_security_info
  Информация о ценной бумаге.

  Args:
    secid - Идентификатор ценной бумаги.
  """

  sample_size = 100
  securities_info = []
  while sample_size >= 100:
    response_security_info = requests.get(
        url=BASE_URL+f"securities/{secid}.json"
    )

    response_status_code = response_security_info.status_code

    if response_status_code == 200:
      boards = response_security_info.json()["boards"]
      security_info = pd.DataFrame(data=boards["data"], columns=boards["columns"])
    else:
      security_info = pd.DataFrame()

    securities_info.append(security_info)
    sample_size = security_info.shape[0]

  return pd.concat(securities_info)

In [211]:
securities_info = pd.concat(securities_data["secid"].apply(get_security_info).to_list()).reset_index(drop=True)

In [212]:
securities_info = securities_info.loc[
    (securities_info["boardid"] == "TQOB") &
    (securities_info["market"] == "bonds") &
    (securities_info["engine"] == "stock") &
    (securities_info["currencyid"] == "RUB"),
    ["secid", "boardid", "market", "engine", "history_from", "history_till"]
].rename(columns={"boardid": "marketprice_boardid"}).reset_index(drop=True)

In [214]:
securities_data = securities_data.merge(securities_info, on=["secid", "marketprice_boardid"], how="left")

In [216]:
securities_data[securities_data["engine"].isna()]

,secid,shortname,regnumber,isin,is_traded,group,marketprice_boardid,market,engine,history_from,history_till
72,SU25065RMFS2,ОФЗ 25065,25065RMFS,RU000A0JQ714,0,stock_bonds,None,NaN,NaN,NaN,NaN
73,SU25067RMFS8,ОФЗ 25067,25067RMFS,RU000A0JQ987,0,stock_bonds,None,NaN,NaN,NaN,NaN
74,SU25068RMFS6,ОФЗ 25068,25068RMFS,RU000A0JQAE3,0,stock_bonds,None,NaN,NaN,NaN,NaN
75,SU25069RMFS4,ОФЗ 25069,25069RMFS,RU000A0JQCM2,0,stock_bonds,None,NaN,NaN,NaN,NaN
76,SU25071RMFS0,ОФЗ 25071,25071RMFS,RU000A0JQLL5,0,stock_bonds,None,NaN,NaN,NaN,NaN
77,SU25072RMFS8,ОФЗ 25072,25072RMFS,RU000A0JQQE9,0,stock_bonds,None,NaN,NaN,NaN,NaN
78,SU25073RMFS6,ОФЗ 25073,25073RMFS,RU000A0JQQW1,0,stock_bonds,None,NaN,NaN,NaN,NaN
80,SU25076RMFS9,ОФЗ 25076,25076RMFS,RU000A0JR779,0,stock_bonds,None,NaN,NaN,NaN,NaN
82,SU25078RMFS5,ОФЗ 25078,25078RMFS,RU000A0JR829,0,stock_bonds,None,NaN,NaN,NaN,NaN
88,SU26198RMFS0,ОФЗ 26198,26198RMFS,RU0001707572,0,stock_bonds,None,NaN,NaN,NaN,NaN
